In [1]:
import jax 
jax.config.update("jax_enable_x64", True)

In [2]:
import cfp
import scanpy as sc
import anndata as ad
import numpy as np
from cfp.external import CFJaxSCVI

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 3
model_name = "polar-jazz-449"

In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{split}.h5ad"
adata_train = sc.read_h5ad(adata_train_path)
adata_test = sc.read_h5ad(adata_test_path)
adata_ood = sc.read_h5ad(adata_ood_path)
    

In [5]:
vae = CFJaxSCVI.load(f"/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024_{split}", adata=adata_train)
adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train)
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test)
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)


INFO     File /lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024_3/model.pt already         
         downloaded                                                                                                


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/model/base/_utils.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU availabl

Epoch 1/23:   0%|          | 0/23 [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/23:   4%|▍         | 1/23 [00:02<01:05,  2.99s/it, v_num=1, train_loss_step=395, train_loss_epoch=395]
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [6]:
adata_ood_ctrl = adata_ood[adata_ood.obs["condition"].str.contains("Vehicle")]
adata_test_ctrl = adata_test[adata_test.obs["condition"].str.contains("Vehicle")]
adata_ood_ctrl.obs["control"] = True
adata_test_ctrl.obs["control"] = True
covariate_data_ood = adata_ood[~adata_ood.obs["condition"].str.contains("Vehicle")].obs.drop_duplicates(subset=["condition"])
covariate_data_test = adata_test[~adata_test.obs["condition"].str.contains("Vehicle")].obs.drop_duplicates(subset=["condition"])

/tmp/ipykernel_1087118/2979151925.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_ood_ctrl.obs["control"] = True
/tmp/ipykernel_1087118/2979151925.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_test_ctrl.obs["control"] = True


In [7]:
cf = cfp.model.CellFlow.load(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/out/{model_name}_CellFlow.pkl")

In [8]:
adata_ood_ctrl

AnnData object with n_obs × n_vars = 1500 × 2001
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5', 'split', '_scvi_batch'
    uns: 'cell_line_dict', 'ecfp_dict', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'ecfp', 'X_scVI'

In [9]:
preds_ood = cf.predict(adata=adata_ood_ctrl, sample_rep="X_scVI", condition_id_key="condition", covariate_data=covariate_data_ood)

/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 174/174 [00:00<00:00, 626.13it/s]


In [13]:
import pandas as pd
all_data = []
conditions = []

for condition, array in preds_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_ood_result = ad.AnnData(X=np.empty((len(all_data_array),2001)), obs=obs_data)
adata_ood_result.obsm["X_scVI_pred"] = all_data_array

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [14]:
adata_ood_result.layers["pred_reconstruction"] = vae.get_reconstructed_expression(adata_ood_result, use_rep="X_scVI_pred")

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [15]:
adata_ood_result.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/sciplex/vae_attention_pooling/adata_ood_with_predictions_{split}.h5ad")


In [ ]:
preds_test = cf.predict(adata=adata_test_ctrl, sample_rep="X_scVI", condition_id_key="condition", covariate_data=covariate_data_test)

/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 509/509 [00:00<00:00, 698.31it/s]


In [ ]:
import pandas as pd
all_data = []
conditions = []

for condition, array in preds_test.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_test_result = ad.AnnData(X=np.empty((len(all_data_array),2000)), obs=obs_data)
adata_test_result.obsm["X_scVI_pred"] = all_data_array

In [ ]:
adata_test_result.layers["pred_reconstruction"] = vae.get_reconstructed_expression(adata_test_result, use_rep="X_scVI_pred")

In [ ]:
adata_test_result.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/sciplex/vae_attention_pooling/adata_test_with_predictions_{split}.h5ad")
